In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # p2.Xlarge only has one GPU

### data preprocessing
https://becominghuman.ai/transfer-learning-retraining-inception-v3-for-custom-image-classification-2820f653c557

### load model (only its conv base, no head) and its weights

In [4]:
from marco import marco

conv_base = marco(include_top=False, input_shape=(599, 599, 3))

In [5]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 599, 599, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_0a_3x3 (Conv2D)          (None, 299, 299, 16) 432         input_2[0][0]                    
__________________________________________________________________________________________________
Conv2d_0a_3x3_BatchNorm (BatchN (None, 299, 299, 16) 48          Conv2d_0a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_0a_3x3_Activation (Activ (None, 299, 299, 16) 0           Conv2d_0a_3x3_BatchNorm[0][0]    
__________________________________________________________________________________________________
Conv2d_1a_

### add new classification layer

In [6]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.AveragePooling2D((8,8), strides=2, padding='valid'))
model.add(layers.Conv2D(10, (1,1), strides=1, padding='same', use_bias=True))
model.add(layers.Flatten())
model.add(layers.Activation('softmax'))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
marco (Model)                (None, 8, 8, 1056)        8959840   
_________________________________________________________________
average_pooling2d_2 (Average (None, 1, 1, 1056)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 10)          10570     
_________________________________________________________________
flatten_2 (Flatten)          (None, 10)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 8,970,410
Trainable params: 8,947,594
Non-trainable params: 22,816
_________________________________________________________________


In [8]:
print('This is the number of trainable weights '
      'before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 192


### freeze conv layer

In [9]:
conv_base.trainable = False

In [10]:
print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights after freezing the conv base: 2


### train model classification layers with data augamentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # resize image
        target_size=(599, 599),
        batch_size=20,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

In [ ]:
model.save('abbvie_marco.h5')

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()